In [1]:
import tensorflow as tf
import keras.backend as K
import trfl
import numpy as np
from keras.layers import Dense, TimeDistributed, Input
from keras.models import Model
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
import gym
import environment.qmaze

Using TensorFlow backend.
/home/jonas/tensorflow/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. E

In [2]:
sess = tf.Session()
K.set_session(sess)

n_envs = 3

inputs = Input(batch_shape = (n_envs, None, 49,))
model = TimeDistributed(Dense(255))(inputs)
policy_logits = TimeDistributed(Dense(4, activation= 'softmax'))(model)
baseline_values = TimeDistributed(Dense(1, activation = None))(model)

sess.run(tf.global_variables_initializer())

e = DummyVecEnv([lambda:gym.make('QMaze-v0') for _ in range(n_envs)])
obs = e.reset()

In [3]:
def act(s_inputs):
    res = sess.run([policy_logits, baseline_values], feed_dict = {
        inputs: np.expand_dims(s_inputs, 1)
    })

    return (res[0][:, 0], res[1][:, 0, 0])

pvalues, bvalues = act(e.reset())

In [4]:
from a2c.a2c import SomeTrainer

t = SomeTrainer()
t._initialize()

Logging to /tmp/openai-2019-02-19-19-41-17-054945


In [5]:
t._predict_single(e.reset())[1].shape

(3,)

In [6]:
batch = t._sample_experience_batch()

In [14]:
r = [(1, ['a','b']), (2, ['4'])]
from functools import reduce
list(map(lambda *x: reduce(lambda a,b:a+b, x), *r))

[3, ['a', 'b', '4']]